In [ ]:
"""
James Gardner
December 2020
"""

import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

In [ ]:
# contant font sizes for plotting
SMALL_SIZE = 12
MEDIUM_SIZE = 16
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def load_and_stitch(filename_tags, path_to_dir = "../data/",
                    filename_header = "SR785_23-12-2020_", plot_true=True, show_true=True, plot_label="",
                    return_data=False):
    """given a list of timestamps, plots the stitched data np array for the full spectrum"""
    # numpy is clever enough to cut the metadata header from each
    data_set = np.array([np.genfromtxt(path_to_dir + filename_header + tag + ".txt") for tag in filename_tags])
    max_freqs = np.array([data[-1,0] for data in data_set])

    # sort in ascending order of max freq
    data_set = data_set[np.argsort(max_freqs)]
    max_freqs = np.sort(max_freqs)

    # stitching
    data_stitched = np.append(data_set[0],
                              np.concatenate([data[data[:,0].searchsorted(max_freqs[i]):]
                                              for i, data in enumerate(data_set[1:])]),
                              axis=0)

    # plotting
    if plot_true:
        yrange = (data_stitched[:,1].min(), data_stitched[:,1].max())

        fig, ax = plt.subplots(figsize=(12,6))
        ax.loglog(data_stitched[:,0], data_stitched[:,1], label=plot_label)
        ax.vlines(max_freqs[:-1], yrange[0], yrange[1], colors=['r'], linestyles=['--'], label='frequency bands')
        ax.set(title="", ylabel=r"absolute value of FFT / $\dfrac{V_{rms}}{\sqrt{Hz}}$", xlabel="frequency / Hz",
               xlim=(None, data_stitched[:,0].max()))
        ax.legend()
        ax.grid(which="both")
        fig.savefig("TL_residual_error_full_spectrum_{0}.pdf".format(filename_tags[0]), bbox_inches="tight")
        if show_true:
            plt.show(fig)
        
    if return_data:
        return max_freqs, data_stitched

In [ ]:
# load_and_stitch(["105934", "110340", "110444", "110640"], plot_label="TL residual error")
# load_and_stitch(["111604", "112046", "112135", "112233"], plot_label="TL dark noise")
# load_and_stitch(["113737", "113841", "113923", "114113"], plot_label="TL jitter + dark noise")

In [ ]:
max_freqs, data1 = load_and_stitch(["105934", "110340", "110444", "110640"], plot_true=False, return_data=True)
max_freqs, data2 = load_and_stitch(["111604", "112046", "112135", "112233"], plot_true=False, return_data=True)
max_freqs, data3 = load_and_stitch(["113737", "113841", "113923", "114113"], plot_true=False, return_data=True)

yrange = (min((data[:,1].min() for data in [data1, data2, data3])),
          max((data[:,1].max() for data in [data1, data2, data3])))

fig, ax = plt.subplots(figsize=(12,6))
ax.loglog(data1[:,0], data1[:,1], label="TL residual error")
ax.loglog(data2[:,0], data2[:,1], label="TL dark noise")
ax.loglog(data3[:,0], data3[:,1], label="TL jitter + dark noise")
ax.vlines(max_freqs[:-1], yrange[0], yrange[1], colors=['r'], linestyles=['--'], label='frequency bands')
ax.set(title="", ylabel=r"absolute value of FFT / $\dfrac{V_{rms}}{\sqrt{Hz}}$", xlabel="frequency / Hz",
       xlim=(None, data1[:,0].max()))
ax.legend()
ax.grid(which="both")
fig.savefig("TL_residual_error_full_spectrum_comparison.pdf", bbox_inches="tight")
fig.savefig("TL_residual_error_full_spectrum_comparison.jpg", bbox_inches="tight")
plt.show(fig)